<a href="https://colab.research.google.com/github/i-am-chen/fianlinclass113-1/blob/main/fianal_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **這個是任何優化器都沒加的程式碼**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Input, Bidirectional, MultiHeadAttention, LayerNormalization
)
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# 資料集載入與處理
# 載入 IMDB 資料集
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# 填充序列到相同長度
maxlen = 200
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post')

# 分割訓練集與驗證集
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 建立 LSTM + Transformer 模型
def create_lstm_transformer_model(vocab_size, embed_dim, lstm_units, transformer_units, num_heads):
    # 輸入層
    inputs = Input(shape=(None,))

    # 嵌入層
    x = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)

    # 雙向 LSTM
    lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True))(x)

    # Transformer 層
    attention_out = MultiHeadAttention(num_heads=num_heads, key_dim=transformer_units)(lstm_out, lstm_out)
    norm_out = LayerNormalization()(attention_out + lstm_out)

    # 平均池化 - 使用 Keras 的 Lambda 層來包裝 tf.reduce_mean
    pooled_out = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(norm_out)

    # 全連接層
    outputs = Dense(1, activation='sigmoid')(pooled_out)

    # 模型
    model = Model(inputs, outputs)
    return model

# 模型參數
vocab_size = 10000
embed_dim = 128
lstm_units = 64
transformer_units = 128
num_heads = 4

# 建立模型
model = create_lstm_transformer_model(vocab_size, embed_dim, lstm_units, transformer_units, num_heads)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 模型摘要
model.summary()

# 訓練模型
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=10,
    batch_size=32
)

# 評估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"測試準確率: {test_acc:.4f}")



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, None, 128)      │      1,280,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, None, 128)      │         98,816 │ embedding_2[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, None, 128)      │        263,808 │ bidirectional_3[0][0], │
│ (MultiHeadAttention)      │                        │                │ bidirectional_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, None, 128)      │              0 │ multi_head_attention_… │
│                           │                        │                │ bidirectional_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, None, 128)      │            256 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 128)            │              0 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │            129 │ lambda_2[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,643,009 (6.27 MB)

 Trainable params: 1,643,009 (6.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.7571 - loss: 0.4759 - val_accuracy: 0.8824 - val_loss: 0.2873
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9303 - loss: 0.1819 - val_accuracy: 0.8800 - val_loss: 0.3072
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9661 - loss: 0.0965 - val_accuracy: 0.8820 - val_loss: 0.3033
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.9839 - loss: 0.0502 - val_accuracy: 0.8782 - val_loss: 0.4170
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9890 - loss: 0.0311 - val_accuracy: 0.8740 - val_loss: 0.4890
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9921 - loss: 0.0248 - val_accuracy: 0.8618 - val_loss: 0.5933
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9944 - loss: 0.0150 - val_accuracy: 0.8696 - val_loss: 0.6389
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9953 - loss: 0.0145 - 